In [1]:
from bce import *
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [2]:
from sklearn.metrics import f1_score, classification_report, precision_score, recall_score

In [3]:
df_dict = data_clean('./bihar_features_gee.csv')

Begin data cleaning.
Read in data from ./bihar_features_gee.csv sucessfully.
All dataframes and labels generated.
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Deleting useless and predictive informative columns
Data cleaning successful.
NaN left: []
NaN left: []
NaN left: []


In [178]:
pred_dict = gdb_machine(df_dict['df'], df_dict['label'], penalty=20, scale=True)

DF Scaling successful.


In [179]:
pred_dict.keys()

dict_keys(['y_score', 'probaility', 'y_test', 'prediction'])

In [182]:
print(classification_report(y_true=pred_dict['y_test'][0], y_pred=pred_dict['prediction'][0]))

             precision    recall  f1-score   support

      False       0.08      0.64      0.15       409
       True       0.98      0.67      0.79      8654

avg / total       0.93      0.67      0.76      9063



In [367]:
def plot_confusion_matrix(pred, truth, title, save_path=None, color=None, normalize=False): 
    print('Plotting confusion matrix.')
    sns.set(font_scale=2)
    try:
        conf_matrix = confusion_matrix(truth, pred)
        if normalize:
            conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
#             print(conf_matrix)
    except:
        raise ValueError('Confuction matrix generation failed.')
    plt.figure(figsize=(5,5))
    ticks = ['', '']
    sns.heatmap(conf_matrix, annot=True, fmt='.1%', cbar=False,
                xticklabels=ticks, 
                yticklabels=ticks,
                annot_kws={"size":40}, 
                square=True, 
                cmap=color)
#     plt.xlabel('Prediction', fontsize=20)
#     plt.ylabel('Truth', fontsize=20)
#     plt.title(title, fontsize=27)
    if save_path==None:
        print('Dispalying the plot.')
        plt.show()
    else:
        print('Saving to %s' % (save_path))
        plt.savefig(save_path)
        plt.close()

In [368]:
colors = ['Oranges', 'Blues', 'Purples']
titles=['All Villages', 'Villages with over 100 Households', 'Villages with over 400 Households']
for i in range(3):
    plot_confusion_matrix(pred=pred_dict['prediction'][i], 
                          truth=pred_dict['y_test'][i], 
                          title='{}'.format(titles[i]), 
                          color=colors[i],
                          normalize=True,
                          save_path='./cunfusion matrix norm {}.png'.format(i))

Plotting confusion matrix.
Saving to ./cunfusion matrix norm 0.png
Plotting confusion matrix.
Saving to ./cunfusion matrix norm 1.png
Plotting confusion matrix.
Saving to ./cunfusion matrix norm 2.png


In [374]:
for i in range(3):
    print(f1_score(pred_dict['prediction'][i], pred_dict['y_test'][i]))

0.7926469579532204
0.6912850812407682
0.803968253968254


In [371]:
# plt.set_facecolor("white")
plt.rcParams["axes.edgecolor"] = "0.15"
plt.rcParams["axes.linewidth"]  = 1.25

In [194]:
1-recall_score(y_pred=pred_dict['prediction'][0], y_true=pred_dict['y_test'][0], pos_label=0)

0.3594132029339854

In [195]:
recall_score(y_pred=pred_dict['prediction'][0], y_true=pred_dict['y_test'][0], pos_label=1)

0.6676681303443495

In [372]:
def plot_roc_curve(pred_dict, filter_list, save_path=None, colors = None):
#     print('Plotting ROC/AUC plot.')
#     plt.style.use('presentation')
#     plt.figure(figsize=(15,15))551A8B
    fig, ax = plt.subplots(figsize=(12,12))
    ax.set_facecolor("white")
    plt.rcParams["axes.edgecolor"] = "0.15"
    plt.rcParams["axes.linewidth"]  = 1.25
    if filter_list == None or len(filter_list) != len(pred_dict['y_test']):
        filter_list = [str(i) for i in range(len(pred_dict['y_test']))]

    counter = 0
    for test, prob, filter_title in zip(pred_dict['y_test'], pred_dict['probaility'], filter_list):
        fpr, tpr, thresholds = roc_curve(test, prob)
        ax.plot(fpr, tpr,label=str(filter_title)+" (AUC ="+str(roc_auc_score(test, prob).round(3))+")", color = colors[counter])
        counter += 1
    
    ax.plot([0, 1], [0, 1], color='lightgrey', lw=2, linestyle='--', label="Chance")
    
    for i in range(3):
        sensitivity = recall_score(y_pred=pred_dict['prediction'][i], y_true=pred_dict['y_test'][i], pos_label=1)
        specificity = recall_score(y_pred=pred_dict['prediction'][i], y_true=pred_dict['y_test'][i], pos_label=0)
        if i == -1:
            ax.plot(1-specificity, sensitivity, marker='o', color='darkblue', ms=10, label="Confusion Matrix")
        else:
            ax.plot(1-specificity, sensitivity, marker='o', color='darkblue', ms=10)
    
    plt.legend(loc=4, fontsize=20, fancybox=True)
    plt.xlabel('False Alarm Rate', fontsize=25)
    plt.ylabel('Correct Detection Rate', fontsize=25)
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.title('Gradient Boosting Classification ROC Curve', fontsize=30)
    if save_path==None:
        print('Dispalying the plot.')
        plt.show()
    else:
        print('Saving to %s' % (save_path))
        plt.savefig(save_path, transparent=True)
        plt.close()

In [373]:
colors = ['#F25F3A', '#3B7DD8', '#551A8B']
plot_roc_curve(pred_dict=pred_dict, 
               filter_list=titles, 
               colors=colors,
               save_path='./roc.png')

Saving to ./roc.png


In [35]:
cm1 = confusion_matrix(pred_dict['prediction'][0], pred_dict['y_test'][0])
sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])

In [4]:
df = data_clean('./bihar_features_gee.csv')

Begin data cleaning.
Read in data from ./bihar_features_gee.csv sucessfully.
All dataframes and labels generated.
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Deleting useless and predictive informative columns
Data cleaning successful.
NaN left: []
NaN left: []
NaN left: []


In [5]:
new_df = df['df'][0].assign(label = df['label'][0])

In [6]:
del df

In [7]:
features = [i for i in new_df.columns if 'DEV' not in i]
cates = ['VIIRS', 'NDVI', 'EVI', 'B', 'POP_DENS']

In [8]:
new_df.head()

,01VIIRS_10th,02VIIRS_10th,03VIIRS_10th,04VIIRS_10th,05VIIRS_10th,06VIIRS_10th,07VIIRS_10th,08VIIRS_10th,09VIIRS_10th,10VIIRS_10th,...,B1_STD_DEV,B10_STD_DEV,B11_STD_DEV,B2_STD_DEV,B3_STD_DEV,B4_STD_DEV,B5_STD_DEV,B6_STD_DEV,B7_STD_DEV,label
0,0.128546,0.112756,0.181673,0.317506,0.324429,0.166385,0.081444,0.330875,0.313948,0.271061,...,0.008436,0.000474,0.000857,0.006898,0.007585,0.007246,0.042781,0.019932,0.010010,False
1,0.171615,0.205894,0.292313,0.594368,0.355164,0.224136,0.075828,0.275970,0.394381,0.371865,...,0.015743,0.001682,0.002017,0.017854,0.023801,0.029072,0.074969,0.043832,0.027392,True
2,0.043099,0.077040,0.170996,0.179324,0.291569,0.161928,0.019642,0.212817,0.301709,0.305066,...,0.023320,0.001737,0.001726,0.027221,0.034742,0.043921,0.086157,0.054006,0.043298,True
3,0.419949,0.381883,0.491086,0.876705,0.542789,0.226300,0.147114,0.297863,0.693142,0.793092,...,0.012286,0.001196,0.001456,0.013462,0.018037,0.019069,0.087061,0.042743,0.023066,True
4,0.204294,0.180164,0.262577,0.376620,0.400297,0.227038,0.006545,0.234676,0.432030,0.329797,...,0.012365,0.001090,0.001329,0.012730,0.016276,0.017785,0.079940,0.036686,0.019230,True


In [9]:
df_1 = new_df[new_df.label == True]
dic_1 = {}
for i in features:
    for j in cates:
        if j in i:
            if j not in dic_1:
                dic_1[j] = []
            dic_1[j].append(np.mean(df_1[i]))
for i in dic_1:
    dic_1[i] = np.mean(dic_1[i])
dic_1

{'B': 0.14532661104327077,
 'EVI': 0.18747993486139472,
 'NDVI': 0.33200667347047663,
 'POP_DENS': 1133.5212350448433,
 'VIIRS': 0.873124998971638}

In [10]:
df_2 = new_df[new_df.label == False]
dic_2 = {}
for i in features:
    for j in cates:
        if j in i:
            if j not in dic_2:
                dic_2[j] = []
            dic_2[j].append(np.mean(df_2[i]))
for i in dic_2:
    dic_2[i] = np.mean(dic_2[i])
dic_2

{'B': 0.14735944544220966,
 'EVI': 0.188167712194117,
 'NDVI': 0.33147480921859807,
 'POP_DENS': 864.1795994373143,
 'VIIRS': 0.6248844423811355}

In [11]:
var = [i for i in new_df.columns if 'DEV' in i]

In [12]:
df_1 = new_df[new_df.label == True]
dic_1_var = {}
for i in var:
    for j in cates:
        if j in i:
            if j not in dic_1_var:
                dic_1_var[j] = []
            dic_1_var[j].append(np.mean(df_1[i]))
for i in dic_1_var:
    dic_1_var[i] = np.mean(dic_1_var[i])
dic_1_var

{'B': 0.019911403941579534,
 'EVI': -0.10682716170618739,
 'NDVI': -0.0977458664563332,
 'POP_DENS': 378.1970465951153,
 'VIIRS': 0.11193612011700373}

In [13]:
df_2 = new_df[new_df.label == False]
dic_2_var = {}
for i in var:
    for j in cates:
        if j in i:
            if j not in dic_2_var:
                dic_2_var[j] = []
            dic_2_var[j].append(np.mean(df_2[i]))
for i in dic_2_var:
    dic_2_var[i] = np.mean(dic_2_var[i])
dic_2_var

{'B': 0.022063287924734522,
 'EVI': -0.09388198017948718,
 'NDVI': -0.08415121169467754,
 'POP_DENS': 326.85558765016594,
 'VIIRS': 0.10152319488100235}

In [14]:
dic_1['POP_DENS'] = dic_1['POP_DENS']/1000
dic_2['POP_DENS'] = dic_2['POP_DENS']/1000
dic_1_var['POP_DENS'] = dic_1_var['POP_DENS']/1000
dic_2_var['POP_DENS'] = dic_2_var['POP_DENS']/1000

In [15]:
bar_width = 0.25
index = np.arange(5)
error_config = {'ecolor': '0.3', 'capsize':5, 'elinewidth':2, 'markeredgewidth':2}
plt.figure(figsize=(10,5))
cates = ['VIIRS', 'NDVI', 'EVI', 'B', 'POP_DENS']
labels = ['Lights at Night', 
          'Green Vegetation', 
          'Enhanced\nGreen Vegetation',
          'Light Spectrum', 
          'Population Density']
# labels = ['VIIRS\nVisible Infrared\nImaging Radiometer Suite\n'+r'$\log(rad)/(cm^2\cdot sr)$', 
#           'NDVI\nNormalized Difference\nVegetation Index\n(-1, 1)', 
#           'EVI\nEnhanced Vegetation Index\n(-1, 1)', 
#           'SB\nSpectral Bands/%', 
#           'Population Density\n'+r'$n(persons)/0.01km^2$']

rects1 = plt.bar(index, [dic_2[cates[i]] for i in index], bar_width,
                 color='skyblue',
                 yerr=[dic_2_var[cates[i]] for i in index],
                 error_kw=error_config,
                 label='Unelectrified')
for i in index:
    plt.text(i-bar_width/2-.01, 
             dic_2[cates[i]] + abs(dic_2_var[cates[i]]) + .05, 
             '%.2f' % dic_2[cates[i]], fontsize=15)
    
rects2 = plt.bar(index + bar_width, [dic_1[cates[i]] for i in index], bar_width,
                 color='lightslategrey',
                 yerr=[dic_1_var[cates[i]] for i in index],
                 error_kw=error_config,
                 label='Electrified')

for i in index:
    plt.text(i+bar_width/2-.01, 
             dic_1[cates[i]] + abs(dic_1_var[cates[i]]) + .05, 
             '%.2f' % dic_1[cates[i]], fontsize=15)

# plt.set_facecolor("white")
# plt.ylim(0,2)
# plt.xlabel('Feature')
plt.ylabel('Average Feature Value')
# plt.title('Normalized Average Feature Value', fontsize=25)
plt.xticks(index + bar_width / 2, labels, fontsize=14)
plt.legend(bbox_to_anchor=(.2, 1))

# rects1.text([1,2,3,4,5])

plt.tight_layout()
# plt.show() 
plt.savefig('./feature avg.png', transparent=True)
plt.close()

In [16]:
import re
gdb_importances = pd.read_csv('./gdb_feature_impor.csv')

In [17]:
gdb_importances = gdb_importances.assign(cate = gdb_importances.names.apply(lambda x: re.findall(r'VIIRS|EVI|POP_DENS|B|NDVI', x)[0]))

In [18]:
cate_dict = {}
for c in gdb_importances.cate.unique():
    cate_dict[c] = [np.mean([i for i in gdb_importances[gdb_importances.cate == c].importance]),
                    np.std([i for i in gdb_importances[gdb_importances.cate == c].importance])]

    
plt.figure(figsize=(12,5))

labels = ['Lights at Night', 
          'Green Vegetation', 
          'Enhanced\nGreen Vegetation',
          'Light Spectrum', 
          'Population Density']

error_config = {'ecolor': '0.3', 'capsize':5, 'elinewidth':2, 'markeredgewidth':2}
plt.bar(index, [cate_dict[i][0] for i in cates], 
        bar_width+.2 ,  # elinewidth=2, # markeredgewidth=2,
        yerr=[cate_dict[i][1] for i in cates], 
        error_kw=error_config)

# plt.errorbar(index, [cate_dict[i][0] for i in cates], 
#              yerr=[cate_dict[i][1] for i in cates], capsize=5,
#              elinewidth=2,
#              markeredgewidth=2)

for i in index:
    plt.text(i-bar_width/2-.05, 
             1.05 * cate_dict[cates[i]][0] + cate_dict[cates[i]][1], 
             '%.4f' % cate_dict[cates[i]][0], fontsize=15)

plt.ylim(0,0.015)
plt.ylabel('Importance Score')
plt.xlabel('Feature')
plt.xticks(index, labels, fontsize=10)
plt.title('Mean Feature Importance')
# plt.show()
plt.savefig('./feature impor.png', transparent=True)
plt.close()

In [266]:
cate_dict

{'B': [0.004973200587162058, 0.00397167158419724],
 'EVI': [0.0038313120132252867, 0.0033187081373105437],
 'NDVI': [0.0029085583121521226, 0.0022066177709323194],
 'POP_DENS': [0.009313007351139985, 0.0017293874748898525],
 'VIIRS': [0.00424094817576092, 0.0032223799044893107]}

In [19]:
new_df.shape

(45311, 369)